In [2]:
import os
from pyspark.sql import SparkSession

In [3]:
input_dir = '../nyt'
output_dir = '../nyt/orc'
os.makedirs(output_dir,exist_ok=True)

In [4]:
#files_to_read = 'ArticlesApril2017'
fn_ls=[fn for fn in os.listdir(input_dir) if '.csv' in fn]

# Load Spark Context

In [5]:
spark = SparkSession.builder.appName('csv2orc').getOrCreate()

# Read and Write Files to orc

In [9]:
for fn in fn_ls:
    input_file_path = os.path.join(input_dir,fn)
    output_file = os.path.join(output_dir,fn[:-4]+'_orc')
    
    df = spark.read.csv(input_file_path, header='true',inferSchema='true')
    df.write.format("orc").save(output_file)

# Test written FIle

In [6]:
output_file = '../nyt/orc/CommentsApril2017_orc'
orc_df = spark.read.orc(output_file)

In [14]:
#orc_df.toPandas()